In [ ]:
%pip install ruprompts 

In [ ]:
!wget https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/train.tsv
!wget https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/dev.tsv

In [ ]:
import pandas as pd

df = pd.read_csv("train.tsv", sep="\t")
df.drop(["index"], axis=1, inplace=True)
df.to_csv("train.tsv", index=False, sep="\t")

## Training

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Load the dataset:

In [ ]:
from datasets import load_dataset

datasets = load_dataset("csv", data_files={"train": "train.tsv", "validation": "dev.tsv"}, sep="\t")
train_dataset = datasets["train"]
valid_dataset = datasets["validation"]

Load the backbone:

In [ ]:
from transformers import GPT2LMHeadModel, AutoTokenizer

backbone_id = "ai-forever/rugpt3large_based_on_gpt2"

model = GPT2LMHeadModel.from_pretrained(backbone_id)
tokenizer = AutoTokenizer.from_pretrained(backbone_id, pad_token="<pad>", eos_token="<pad>")

Define the prompt format:

In [ ]:
from ruprompts import PromptFormat

prompt_format = PromptFormat("<P*100>{toxic_comment}<P*20>")

Define the parametrization of trainable embeddings:

In [ ]:
from ruprompts import TensorPromptProvider
from transformers import set_seed

set_seed(1)

prompt_provider = TensorPromptProvider()

Compose prompt format and prompt provider into prompt object and apply it to the model and tokenizer, i.e. add special tokens to the tokenizer and modify the layer of input embeddings of the model:

In [ ]:
from ruprompts import Prompt

prompt = Prompt(prompt_format, prompt_provider)
prompt.patch(model, tokenizer)

Preprocess the data:
1. format the data entries with the specified prompt format
2. tokenize the resulting sequences
3. truncate the `truncation_field` if sequence length exceeds `max_tokens`

In [ ]:
from ruprompts import Text2TextPreprocessor

preprocessor = Text2TextPreprocessor(
    prompt_format=prompt_format,
    tokenizer=tokenizer,
    target_field="neutral_comment1",
    max_tokens=1792,
    truncation_field="toxic_comment",
)

train_dataset = train_dataset.map(preprocessor)
valid_dataset = valid_dataset.map(preprocessor)

Define training arguments:

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=".",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,
    eval_steps=1000,
    save_steps=1000,
    logging_steps=100,
    evaluation_strategy="steps",
    save_strategy="steps",
    logging_strategy="steps",
    save_total_limit=2,
    metric_for_best_model="eval_loss",
    learning_rate=0.1,
    max_steps=100000,
    report_to="tensorboard",
    # report_to=["tensorboard", "wandb"],  # uncomment to log to WandB
    logging_dir="logs",
    seed=1,
)

Choose optimization options:

In [ ]:
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(prompt_provider.parameters(), lr=training_args.learning_rate)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=2000,
    num_training_steps=training_args.max_steps,
)

Define the callbacks and start training:

In [ ]:
from transformers import Trainer
from ruprompts.callbacks import (
    FreezeTransformerUnfreezePrompt,
    ReduceCheckpoint,
    SavePretrainedPrompt,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=preprocessor.collate_fn(),
    optimizers=(optimizer, scheduler),
    callbacks=[FreezeTransformerUnfreezePrompt(), ReduceCheckpoint(), SavePretrainedPrompt(prompt)],
)

trainer.train()

## Inference

Load prompt from the last checkpoint:

In [ ]:
from transformers import pipeline

prompt = Prompt.from_pretrained(f"./checkpoint-{training_args.max_steps}")

ppln = pipeline("text2text-generation-with-prompt", prompt=prompt, model=model, tokenizer=tokenizer, device=0)

In [ ]:
ppln({"toxic_comment": "Ублюдок, мать твою, а ну иди сюда"}, do_sample=False)

Run inference:

In [ ]:
from tqdm import tqdm
import transformers

transformers.logging.set_verbosity_error()

beam_count = 10

predictions = []
    
for i in tqdm(valid_dataset["toxic_comment"]):
    options = ppln(
        {"toxic_comment": i},
        do_sample=False,
        num_beams=beam_count,
        num_return_sequences=beam_count,
    )

    options = [i["generated_text"].replace("<pad>", "") for i in options]
    answer = sorted(options, key=len)[-1]  # get longest answer
    predictions.append(answer)

with open("subm.txt", "w") as f:
    f.writelines(list(map(lambda x: x.replace("\n", " ") + "\n", predictions)))